### **Train**

After using copy paste technique on around 18000 documents, we will start training our checkbox detector using Ultralytics' YOLOv8 model. To accelerate the training process, we will migrate the training process onto Kaggle and use GPU-P100 accelerator. 

**Download necessary files and weights**

Install ultralytics

In [ ]:
%pip install -U ultralytics

Download the model and dataset information files and save them in the appropriate folders

In [ ]:
!wget -P /kaggle/working/ultralytics/ultralytics/yolo https://github.com/ultralytics/assets/releases/download/v0.0.0/yolov8s.pt

In [ ]:
!wget -P /kaggle/working/ultralytics/ultralytics/datasets https://raw.githubusercontent.com/LynnHaDo/Checkbox-Detection/main/all-boxes.yaml

Save model backbone config

In [ ]:
!wget -P /kaggle/working/ultralytics/ultralytics/models/v8 https://raw.githubusercontent.com/LynnHaDo/Checkbox-Detection/main/yolov8s.yaml

Update the path to dataset using ruamel package

In [ ]:
%pip install ruamel.yaml

In [ ]:
# Edit boxes.yaml file
box_yaml = '/kaggle/working/ultralytics/ultralytics/datasets/all-boxes.yaml'
config, ind, bsi = ruamel.yaml.util.load_yaml_guess_indent(open(box_yaml))

config['path'] = '/kaggle/input/rvl-all-classes/rvl' # Change the path to dataset 

yaml = ruamel.yaml.YAML()
yaml.indent(mapping=ind, sequence=ind, offset=bsi) 
with open(box_yaml, 'w') as fp:
    yaml.dump(config, fp)

**Start training**

Check for CUDA version

In [ ]:
import torch
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)

Check the available GPU devices

In [ ]:
import subprocess
import pprint

sp = subprocess.Popen(['nvidia-smi'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

out_str = sp.communicate()
out_list = str(out_str[0]).split('\\n')

out_dict = {}

for item in out_list:
    print(item)

To prevent any unexpected stops in the training process on Kaggle, we might want to disable wandb 

In [ ]:
!wandb disabled

In [ ]:
from ultralytics import YOLO
# Due to lack of computational resources, we will use YOLOv8s model with to train the initial checkpoint 
model = YOLO("/kaggle/working/ultralytics/ultralytics/models/v8/yolov8s.yaml").load("/kaggle/working/ultralytics/ultralytics/yolo/yolov8s.pt")
# Train
model.train(data="/kaggle/working/ultralytics/ultralytics/datasets/all-boxes.yaml",
            epochs=200,
            patience=75,
            imgsz=640,
            workers=8,
            batch=-1,
            name="doclaynet_base_train1",
            mixup=0.3,
            copy_paste=0.3,
            device=0)
